In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 이미지 정규화 및 텐서 변환
transform = transforms.Compose([
    transforms.ToTensor(), # 0~255를 0-1로
    transforms.Normalize((0.5,), (0.5,)) # 정규화 -> -1~+1
])

# 데이터셋 다운로드
train_dataset = datasets.MNIST(root='./data',
                               train = True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data',
                               train = False, download=True, transform=transform)

# 데이터로더
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv_layer = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size = 3, padding = 1), # 28x28 -> 28x28
        nn.ReLU(),
        nn.MaxPool2d(2), #28x28 -> 14x14
        nn.Conv2d(32, 64, kernel_size = 3, padding = 1), #14x14 -> 14x14
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.fc_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64 * 7 * 7, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    x = self.conv_layer(x)
    x = self.fc_layer(x)
    return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from torchsummary import summary
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          18,496
              ReLU-5           [-1, 64, 14, 14]               0
         MaxPool2d-6             [-1, 64, 7, 7]               0
           Flatten-7                 [-1, 3136]               0
            Linear-8                   [-1, 64]         200,768
              ReLU-9                   [-1, 64]               0
           Linear-10                   [-1, 10]             650
Total params: 220,234
Trainable params: 220,234
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.67
Params size (MB): 0.84
Estimated T

In [ ]:
for epoch in range(5): # 64배치 5에포크 train 시행
  model.train() # train mode 시행
  running_loss = 0.0 # 현재 에포크에서 전체 손실 합계 변수 초기화
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device) # 데이터를 학습 장치로 이동
    outputs = model(images) # 결과 확인
    loss = criterion(outputs, labels) # 손실 계산
    optimizer.zero_grad() # gradient 초기화
    loss.backward() # 역전파 시행. 손실 기준으로 각 파라미터에 대한 gradient 계산
    optimizer.step()
    running_loss += loss.item() # 현재 배치의 loss를 누적

  print(f'Epoch {epoch +1}, Loss: {running_loss / len(train_loader):.4f}')


Epoch 1, Loss: 0.0528
Epoch 2, Loss: 0.0370
Epoch 3, Loss: 0.0303
Epoch 4, Loss: 0.0216
Epoch 5, Loss: 0.0185


In [ ]:
model.eval()
correct = 0; total = 0

with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1) # 예측값의 클래스 추출

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test ACC: {accuracy:.2f}%")

Test ACC: 99.00%
